# Práctica 5 - Visión por Computador

### Autores

- Pablo Segura López
- Kilian Armas Pérez

## Tarea

In [1]:
import cv2
import time
import os
os.chdir("./Material P5")
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

In [ ]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)

# Face detection and eye model setup (DNN + DLIB68)
imodoF = 2
imodoE = 1

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    if ret:
        # For HS normalization
        B, G, R = cv2.split(frame)

        # Search face with a specific setup for face and eye detection
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
        if values is not None:
            face, eyes, shape = values

            #draws face container
            [x, y , w, h] = face
            if x > -1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # draws eyes and mask if available
                [lex, ley, rex, rey] = eyes
                if lex > -1:
                    # Show detected facial elements
                    if imodoF > 0:
                        for (x, y) in shape:
                            cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                    cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                    cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                    # Normalize and show
                    # color channels
                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Bnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Gnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Rnorm = normalizatorHS.normf_image
                    NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                    cv2.imshow("Normalized", NormBGR)

        # Show resulting image
        cv2.putText(frame, f"Points: 0", (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Cam', frame)
        
        # Esc to finish
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc
            break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()